In [10]:
import sqlite3
import typer
from rich import print
import requests

In [11]:
# Send a GET request to the URL
url = 'https://www.londonambulance.nhs.uk/working-for-us/volunteering-with-us/emergency-responders/'
response = requests.get(url)

In [12]:
if response.status_code == 200:
    # Return the HTML content of the page
    print(response.text[:500])
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")


<!DOCTYPE html>

<!-->
    <html class="lt-ie9" lang="en">
    <!-->
<!-->
        <!-->
<html lang="en">
<!--<!-->

<head>

    <meta name="viewport" content="width=device-width, initial-scale=1">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <style>img:is(, ) { contain-intrinsic-size: 3000px 1500px }</style>
        
                <!-- All in One SEO 4.8.0 - aioseo.com -->
        <meta name="description" content="Emergency Responders (E

In [13]:
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
import nltk
from nltk.tokenize import sent_tokenize

# nltk.download('reuters')
# nltk.download('punkt')
# nltk.download('punkt_tab')

In [14]:
# Get the raw text from the reuters corpus
raw_text = ' '.join(reuters.raw(fileid) for fileid in reuters.fileids())

# Tokenize the raw text into sentences
sentences = sent_tokenize(raw_text)

sentences[:5]

["ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT\n  Mounting trade friction between the\n  U.S. And Japan has raised fears among many of Asia's exporting\n  nations that the row could inflict far-reaching economic\n  damage, businessmen and officials said.",
 'They told Reuter correspondents in Asian capitals a U.S.\n  Move against Japan might boost protectionist sentiment in the\n  U.S. And lead to curbs on American imports of their products.',
 "But some exporters said that while the conflict would hurt\n  them in the long-run, in the short-term Tokyo's loss might be\n  their gain.",
 "The U.S. Has said it will impose 300 mln dlrs of tariffs on\n  imports of Japanese electronics goods on April 17, in\n  retaliation for Japan's alleged failure to stick to a pact not\n  to sell semiconductors on world markets at below cost.",
 'Unofficial Japanese estimates put the impact of the tariffs\n  at 10 billion dlrs and spokesmen for major electronics firms\n  said they would virtually halt 

In [15]:
# Create a placeholder for model
model2 = defaultdict(lambda: defaultdict(lambda: 0))
model3 = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of co-occurance  
for sentence in reuters.sents():
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        model2[(w1, w2)][w3] += 1
    for w1, w2, w3, w4 in nltk.ngrams(sentence, 4, pad_right=True, pad_left=True):
        model3[(w1, w2, w3)][w4] += 1

# Let's transform the counts to probabilities
for w1_w2 in model2:
    total_count = float(sum(model2[w1_w2].values()))
    for w3 in model2[w1_w2]:
        model2[w1_w2][w3] /= total_count

for w1_w2_w3 in model3:
    total_count = float(sum(model3[w1_w2_w3].values()))
    for w4 in model3[w1_w2_w3]:
        model3[w1_w2_w3][w4] /= total_count

print(dict(model2['today', 'the']))
print(dict(model3['today', 'the', 'company']))


{
    'public': 0.05555555555555555,
    'European': 0.05555555555555555,
    'Bank': 0.05555555555555555,
    'price': 0.1111111111111111,
    'emirate': 0.05555555555555555,
    'overseas': 0.05555555555555555,
    'newspaper': 0.05555555555555555,
    'company': 0.16666666666666666,
    'Turkish': 0.05555555555555555,
    'increase': 0.05555555555555555,
    'options': 0.05555555555555555,
    'Higher': 0.05555555555555555,
    'pound': 0.05555555555555555,
    'Italian': 0.05555555555555555,
    'time': 0.05555555555555555
}

{'hoped': 0.3333333333333333, 'announced': 0.3333333333333333, 'proposed': 0.3333333333333333}

In [16]:
import random

# starting words
text = ["today", "the"]
sentence_finished = False

while not sentence_finished:
    # select a random probability threshold  
    r = random.random()
    accumulator = .0
    word_found = False

    for word in model3[tuple(text[-3:])].keys():
        accumulator += model3[tuple(text[-3:])][word]
        # select words that are above the probability threshold
        if accumulator >= r:
            text.append(word)
            word_found = True
            break

    if not word_found:
        print('No word found')
        for word in model2[tuple(text[-2:])].keys():
            accumulator += model2[tuple(text[-2:])][word]
            # select words that are above the probability threshold
            if accumulator >= r:
                text.append(word)
                break

    if text[-2:] == [None, None]:
        sentence_finished = True
    
print (' '.join([t for t in text if t]))

No word found

today the increase was entirely explained by world market conditions .

In [8]:
data_text = """The unanimous Declaration of the thirteen united States of America, When in the Course of human events, it becomes necessary for one people to dissolve the political bands which have connected them with another, and to assume among the powers of the earth, the separate and equal station to which the Laws of Nature and of Nature's God entitle them, a decent respect to the opinions of mankind requires that they should declare the causes which impel them to the separation.

We hold these truths to be self-evident, that all men are created equal, that they are endowed by their Creator with certain unalienable Rights, that among these are Life, Liberty and the pursuit of Happiness.--That to secure these rights, Governments are instituted among Men, deriving their just powers from the consent of the governed, --That whenever any Form of Government becomes destructive of these ends, it is the Right of the People to alter or to abolish it, and to institute new Government, laying its foundation on such principles and organizing its powers in such form, as to them shall seem most likely to effect their Safety and Happiness. Prudence, indeed, will dictate that Governments long established should not be changed for light and transient causes; and accordingly all experience hath shewn, that mankind are more disposed to suffer, while evils are sufferable, than to right themselves by abolishing the forms to which they are accustomed. But when a long train of abuses and usurpations, pursuing invariably the same Object evinces a design to reduce them under absolute Despotism, it is their right, it is their duty, to throw off such Government, and to provide new Guards for their future security.--Such has been the patient sufferance of these Colonies; and such is now the necessity which constrains them to alter their former Systems of Government. The history of the present King of Great Britain is a history of repeated injuries and usurpations, all having in direct object the establishment of an absolute Tyranny over these States. To prove this, let Facts be submitted to a candid world.

He has refused his Assent to Laws, the most wholesome and necessary for the public good.

He has forbidden his Governors to pass Laws of immediate and pressing importance, unless suspended in their operation till his Assent should be obtained; and when so suspended, he has utterly neglected to attend to them.

He has refused to pass other Laws for the accommodation of large districts of people, unless those people would relinquish the right of Representation in the Legislature, a right inestimable to them and formidable to tyrants only.

He has called together legislative bodies at places unusual, uncomfortable, and distant from the depository of their public Records, for the sole purpose of fatiguing them into compliance with his measures.

He has dissolved Representative Houses repeatedly, for opposing with manly firmness his invasions on the rights of the people.

He has refused for a long time, after such dissolutions, to cause others to be elected; whereby the Legislative powers, incapable of Annihilation, have returned to the People at large for their exercise; the State remaining in the mean time exposed to all the dangers of invasion from without, and convulsions within.

He has endeavoured to prevent the population of these States; for that purpose obstructing the Laws for Naturalization of Foreigners; refusing to pass others to encourage their migrations hither, and raising the conditions of new Appropriations of Lands.

He has obstructed the Administration of Justice, by refusing his Assent to Laws for establishing Judiciary powers.

He has made Judges dependent on his Will alone, for the tenure of their offices, and the amount and payment of their salaries.

He has erected a multitude of New Offices, and sent hither swarms of Officers to harrass our people, and eat out their substance.

He has kept among us, in times of peace, Standing Armies without the Consent of our legislatures.

He has affected to render the Military independent of and superior to the Civil power.

He has combined with others to subject us to a jurisdiction foreign to our constitution, and unacknowledged by our laws; giving his Assent to their Acts of pretended Legislation:

For Quartering large bodies of armed troops among us:

For protecting them, by a mock Trial, from punishment for any Murders which they should commit on the Inhabitants of these States:

For cutting off our Trade with all parts of the world:

For imposing Taxes on us without our Consent:

For depriving us in many cases, of the benefits of Trial by Jury:

For transporting us beyond Seas to be tried for pretended offences

For abolishing the free System of English Laws in a neighbouring Province, establishing therein an Arbitrary government, and enlarging its Boundaries so as to render it at once an example and fit instrument for introducing the same absolute rule into these Colonies:

For taking away our Charters, abolishing our most valuable Laws, and altering fundamentally the Forms of our Governments:

For suspending our own Legislatures, and declaring themselves invested with power to legislate for us in all cases whatsoever.

He has abdicated Government here, by declaring us out of his Protection and waging War against us.

He has plundered our seas, ravaged our Coasts, burnt our towns, and destroyed the lives of our people.

He is at this time transporting large Armies of foreign Mercenaries to compleat the works of death, desolation and tyranny, already begun with circumstances of Cruelty & perfidy scarcely paralleled in the most barbarous ages, and totally unworthy the Head of a civilized nation.

He has constrained our fellow Citizens taken Captive on the high Seas to bear Arms against their Country, to become the executioners of their friends and Brethren, or to fall themselves by their Hands.

He has excited domestic insurrections amongst us, and has endeavoured to bring on the inhabitants of our frontiers, the merciless Indian Savages, whose known rule of warfare, is an undistinguished destruction of all ages, sexes and conditions.

In every stage of these Oppressions We have Petitioned for Redress in the most humble terms: Our repeated Petitions have been answered only by repeated injury. A Prince whose character is thus marked by every act which may define a Tyrant, is unfit to be the ruler of a free people.

Nor have We been wanting in attentions to our Brittish brethren. We have warned them from time to time of attempts by their legislature to extend an unwarrantable jurisdiction over us. We have reminded them of the circumstances of our emigration and settlement here. We have appealed to their native justice and magnanimity, and we have conjured them by the ties of our common kindred to disavow these usurpations, which, would inevitably interrupt our connections and correspondence. They too have been deaf to the voice of justice and of consanguinity. We must, therefore, acquiesce in the necessity, which denounces our Separation, and hold them, as we hold the rest of mankind, Enemies in War, in Peace Friends.

We, therefore, the Representatives of the united States of America, in General Congress, Assembled, appealing to the Supreme Judge of the world for the rectitude of our intentions, do, in the Name, and by Authority of the good People of these Colonies, solemnly publish and declare, That these United Colonies are, and of Right ought to be Free and Independent States; that they are Absolved from all Allegiance to the British Crown, and that all political connection between them and the State of Great Britain, is and ought to be totally dissolved; and that as Free and Independent States, they have full Power to levy War, conclude Peace, contract Alliances, establish Commerce, and to do all other Acts and Things which Independent States may of right do. And for the support of this Declaration, with a firm reliance on the protection of divine Providence, we mutually pledge to each other our Lives, our Fortunes and our sacred Honor."""

In [9]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [10]:
import re

def text_cleaner(text):
    # lower case text
    newString = text.lower()
    newString = re.sub(r"'s\b","",newString)
    # remove punctuations
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    long_words=[]
    # remove short word
    for i in newString.split():
        if len(i)>=3:                  
            long_words.append(i)
    return (" ".join(long_words)).strip()

# preprocess the text
data_new = text_cleaner(data_text)

In [11]:
def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        # select sequence of tokens
        seq = text[i-length:i+1]
        # store
        sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences

# create sequences   
sequences = create_seq(data_new)

Total Sequences: 7052


In [13]:
# create a character mapping index
chars = sorted(list(set(data_new)))
mapping = dict((c, i) for i, c in enumerate(chars))

def encode_seq(seq):
    sequences = list()
    for line in seq:
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)
    return sequences

# encode the sequences
sequences = encode_seq(sequences)

In [15]:
from sklearn.model_selection import train_test_split

# vocabulary size
vocab = len(mapping)
sequences = np.array(sequences)
# create X and y
X, y = sequences[:,:-1], sequences[:,-1]
# one hot encode y
y = to_categorical(y, num_classes=vocab)
# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Train shape: (6346, 30) Val shape: (706, 30)


In [ ]:
# define model
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())

# compile the model
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
# fit the model
model.fit(X_tr, y_tr, epochs=100, verbose=2, validation_data=(X_val, y_vxal))

/Users/elenafaillace/Documents/all_code/LM-links-app/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/100
199/199 - 7s - 37ms/step - acc: 0.2140 - loss: 2.7157 - val_acc: 0.3229 - val_loss: 2.3918
Epoch 2/100
199/199 - 6s - 31ms/step - acc: 0.3301 - loss: 2.2801 - val_acc: 0.3442 - val_loss: 2.2457
Epoch 3/100
199/199 - 6s - 31ms/step - acc: 0.3612 - loss: 2.1555 - val_acc: 0.3669 - val_loss: 2.1527
Epoch 4/100
199/199 - 6s - 29ms/step - acc: 0.3854 - loss: 2.0562 - val_acc: 0.3909 - val_loss: 2.0874
Epoch 5/100
199/199 - 6s - 29ms/step - acc: 0.4092 - loss: 1.9762 - val_acc: 0.4136 - val_loss: 2.0267
Epoch 6/100
199/199 - 6s - 30ms/step - acc: 0.4338 - loss: 1.8940 - val_acc: 0.4433 - val_loss: 1.9926
Epoch 7/100
199/199 - 6s - 30ms/step - acc: 0.4502 - loss: 1.8153 - val_acc: 0.4575 - val_loss: 1.9643
Epoch 8/100
199/199 - 6s - 30ms/step - acc: 0.4754 - loss: 1.7385 - val_acc: 0.4632 - val_loss: 1.9249
Epoch 9/100
199/199 - 6s - 30ms/step - acc: 0.4931 - loss: 1.6745 - val_acc: 0.4646 - val_loss: 1.8951
Epoch 10/100
199/199 - 6s - 30ms/step - acc: 0.5203 - loss: 1.6012 -